In [1]:
import xarray as xr 
import logging
import datetime 
import pandas as pd 

In [2]:
def read_qpe_netcdf(file_path):
    """
    Read WRNZ QPE NetCDF file and return xarray Dataset of rain rate with:
    - 'rain' variable in [time, yc, xc] order
    - time as timezone-aware UTC datetimes
    - EPSG:2193 (NZTM2000) projection info added using CF conventions
    - Return None on error reading the file

    Assumes that the input file is rain rate in [t,y,x] order
    """

    try:
        ds = xr.open_dataset(file_path, decode_cf=True, mask_and_scale=True)
        ds.load()

        # Make the times timezone-aware UTC
        time_values = ds["time"].values.astype("datetime64[ns]")
        time_utc = pd.DatetimeIndex(time_values, tz=datetime.UTC)
        ds["time"] = ("time", time_utc)

        # Rename
        ds = ds.rename({"rainfall": "rain"})

        # Define CF-compliant grid mapping for EPSG:2193
        crs = xr.DataArray(
            0,
            attrs={
                "grid_mapping_name": "transverse_mercator",
                "scale_factor_at_central_meridian": 0.9996,
                "longitude_of_central_meridian": 173.0,
                "latitude_of_projection_origin": 0.0,
                "false_easting": 1600000.0,
                "false_northing": 10000000.0,
                "semi_major_axis": 6378137.0,
                "inverse_flattening": 298.257222101,
                "spatial_ref": "EPSG:2193",
            },
            name="NZTM2000",
        )

        ds["NZTM2000"] = crs
        ds["rain"].attrs["grid_mapping"] = "NZTM2000"

        ds = ds[["rain", "NZTM2000"]]
        ds = ds.assign_coords(time=ds["time"], yc=ds["y"], xc=ds["x"])

        return ds

    except (ValueError, OverflowError, TypeError) as e:
        logging.warning(f"Failed to read {file_path}: {e}")
        return None


In [4]:
file_name = "/home/alanseed/alan/nowcast_demo/data/akl_rad_qpe_20230127.nc"
try:
    ds = xr.load_dataset(file_name, decode_cf=True, mask_and_scale=True)
except (ValueError, OverflowError, TypeError) as e:
    logging.warning(f"Failed to read {file_name}: {e}")
    
print(ds.info())


xarray.Dataset {
dimensions:
	time = 144 ;
	y = 256 ;
	x = 256 ;

variables:
	float64 rainfall(time, y, x) ;
		rainfall:units = mm/h ;
		rainfall:long_name = Rainfall rate ;
		rainfall:grid_mapping = projection ;
	int32 projection() ;
		projection:crs_wkt = PROJCRS["NZGD2000 / New Zealand Transverse Mercator 2000",BASEGEOGCRS["NZGD2000",DATUM["New Zealand Geodetic Datum 2000",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4167]],CONVERSION["New Zealand Transverse Mercator 2000",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",173,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",1600000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False north